In [1]:
from pyspark.sql import functions as F 
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.storagelevel import StorageLevel

In [2]:
list1=[[1,"""{"elements": [{{"sev": {"imp":"M","TIME":"20","min_lim":"10"}},{"sev": {"imp":"H","TIME":"10","min_lim":"5","max_lim":"10"}},{"sev": {"imp":"C","TIME":"5","min_lim":"1","max_lim":"4"}}}]}"""]]

#list2=[[1,[{"elements": [{{"sev": {"imp":"M","TIME":"20","min_lim":"10"}},{"sev": {"imp":"H","TIME":"10","min_lim":"5","max_lim":"10"}},{"sev": {"imp":"C","TIME":"5","min_lim":"1","max_lim":"4"}}}]}]]]

df=spark.createDataFrame(list1,['ID','data'])
#df1=spark.createDataFrame(list2,['ID','data'])

df.show()

+---+--------------------+
 ID| data|
+---+--------------------+
 1|{"elements": [{{"...|
+---+--------------------+

In [3]:
df.select(F.schema_of_json('{"elements": [{{"sev": {"imp" : "M", "TIME" : "20","min_lim" : "10"}}}]}')).collect()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-2112672745328028> in <module> 
 ----> 1 df . select ( F . schema_of_json ( '{"elements": [{{"sev": {"imp" : "M", "TIME" : "20","min_lim" : "10"}}}]}' ) ) . collect ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in collect (self) 
 550 # Default path used in OSS Spark / for non-DF-ACL clusters: 
 551 with SCCallSiteSync ( self . _sc ) as css : 
 --> 552 sock_info = self . _jdf . collectToPython ( ) 
 553 return list ( _load_from_socket ( sock_info , BatchedSerializer ( PickleSerializer ( ) ) ) ) 
 554 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o1870.collectToPython.
: com.fasterxml.jackson.core.JsonParseException: Unexpected character ('{' (code 123)): was expecting double-quote to start field name
 at [Source: java.io.InputStreamReader@5ff706b1; line: 1, column: 17]
	at com.fasterxml.jackson.core.JsonParser._constructError(JsonParser.java:1581)
	at com.fasterxml.jackson.core.base.ParserMinimalBase._reportError(ParserMinimalBase.java:533)
	at com.fasterxml.jackson.core.base.ParserMinimalBase._reportUnexpectedChar(ParserMinimalBase.java:462)
	at com.fasterxml.jackson.core.json.ReaderBasedJsonParser._handleOddName(ReaderBasedJsonParser.java:1502)
	at com.fasterxml.jackson.core.json.ReaderBasedJsonParser.nextToken(ReaderBasedJsonParser.java:624)
	at org.apache.spark.sql.catalyst.json.JacksonUtils$.nextUntil(JacksonUtils.scala:29)
	at org.apache.spark.sql.catalyst.json.JsonInferSchema$.inferField(JsonInferSchema.scala:134)
	at org.apache.spark.sql.catalyst.json.JsonInferSchema$.inferField(JsonInferSchema.scala:152)
	at org.apache.spark.sql.catalyst.json.JsonInferSchema$.inferField(JsonInferSchema.scala:137)
	at org.apache.spark.sql.catalyst.expressions.SchemaOfJson$$anonfun$9.apply(jsonExpressions.scala:764)
	at org.apache.spark.sql.catalyst.expressions.SchemaOfJson$$anonfun$9.apply(jsonExpressions.scala:762)
	at org.apache.spark.util.Utils$.tryWithResource(Utils.scala:2787)
	at org.apache.spark.sql.catalyst.expressions.SchemaOfJson.eval(jsonExpressions.scala:762)
	at org.apache.spark.sql.catalyst.optimizer.ConstantFolding$$anonfun$apply$1$$anonfun$applyOrElse$1.applyOrElse(expressions.scala:52)
	at org.apache.spark.sql.catalyst.optimizer.ConstantFolding$$anonfun$apply$1$$anonfun$applyOrElse$1.applyOrElse(expressions.scala:45)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$2.apply(TreeNode.scala:279)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$2.apply(TreeNode.scala:279)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformDown(TreeNode.scala:278)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$3.apply(TreeNode.scala:284)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$3.apply(TreeNode.scala:284)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChi

In [4]:
list=[[[1,1,1,1,1,2,2,1,1,1,1,1,1],[1,2,3,4,5,6,7,8,9,10,11,12,13]]]


df=spark.createDataFrame(list,['value','day'])

df.show()

+--------------------+--------------------+
 value| day|
+--------------------+--------------------+
[1, 1, 1, 1, 1, 2...|[1, 2, 3, 4, 5, 6...|
+--------------------+--------------------+

In [5]:
df=df.withColumn("zipped", F.explode(F.arrays_zip("value","day"))).select("zipped.*")

df.show()

+-----+---+
value|day|
+-----+---+
 1| 1|
 1| 2|
 1| 3|
 1| 4|
 1| 5|
 2| 6|
 2| 7|
 1| 8|
 1| 9|
 1| 10|
 1| 11|
 1| 12|
 1| 13|
+-----+---+

In [6]:
w1=Window().orderBy("day")
df.withColumn("lag", F.when(F.lag("value").over(w1)!=F.col("value"), F.lit(1)).otherwise(F.lit(0)))\
  .withColumn("group", F.sum("lag").over(w1) + 1).drop("lag").show()

+-----+---+-----+
value|day|group|
+-----+---+-----+
 1| 1| 1|
 1| 2| 1|
 1| 3| 1|
 1| 4| 1|
 1| 5| 1|
 2| 6| 2|
 2| 7| 2|
 1| 8| 3|
 1| 9| 3|
 1| 10| 3|
 1| 11| 3|
 1| 12| 3|
 1| 13| 3|
+-----+---+-----+

In [7]:
list=[['Apple','A'],
      ['Google','G'],
      ['Facebook','F']]

df=spark.createDataFrame(list,['col1','col2'])

df.show()

+--------+----+
 col1|col2|
+--------+----+
 Apple| A|
 Google| G|
Facebook| F|
+--------+----+

In [8]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
w=Window().orderBy("col3")
arr=["SFO","LA","NYC"]

df.withColumn("col3", F.array(*[F.lit(x) for x in arr]))\
  .withColumn("rownum", F.row_number().over(w))\
  .withColumn("col3", F.expr("""element_at(col3,rownum)""")).drop("rownum").show()

+--------+----+----+
 col1|col2|col3|
+--------+----+----+
 Apple| A| SFO|
 Google| G| LA|
Facebook| F| NYC|
+--------+----+----+

In [9]:
list=[[[{"service_id": "S121", "price": 1256}, {"service_id": "S022", "price": 1149}],'a01','2020-01-17'],
       [[{"service_id": "S121", "price": 1256}, {"service_id": "S022", "price": 1149}],'a01','2020-01-17']]

df=spark.createDataFrame(list,['contents','account_id','date'])
df.show(truncate=False) 
df.printSchema()

+--------------------------------------------------------------------------+----------+----------+
contents |account_id|date |
+--------------------------------------------------------------------------+----------+----------+
[[price -> 1256, service_id -> S121], [price -> 1149, service_id -> S022]]|a01 |2020-01-17|
[[price -> 1256, service_id -> S121], [price -> 1149, service_id -> S022]]|a01 |2020-01-17|
+--------------------------------------------------------------------------+----------+----------+

root
-- contents: array (nullable = true)
 |-- element: map (containsNull = true)
 | |-- key: string
 | |-- value: string (valueContainsNull = true)
-- account_id: string (nullable = true)
-- date: string (nullable = true)

In [10]:
df.withColumn("keys",F.explode(F.expr("""transform(contents, x->map_values(x))""")))\
  .withColumn("price", F.col("keys")[0])\
  .withColumn("service_id", F.col("keys")[1]).drop("keys","contents").show(truncate=False)

+----------+----------+-----+----------+
account_id|date |price|service_id|
+----------+----------+-----+----------+
a01 |2020-01-17|1256 |S121 |
a01 |2020-01-17|1149 |S022 |
a01 |2020-01-17|1256 |S121 |
a01 |2020-01-17|1149 |S022 |
+----------+----------+-----+----------+

In [11]:
list=[[1,1,'ABC','ABC'],
      [2,1,'ABC','ABC'],
      [3,6,'ABCDE','ABCDE'],
      [4,3,'ABCDE','ABC'],
      [5,7,'ABCDE','ABC']]

df=spark.createDataFrame(list,['TranID','RevID','CheckingCol','CheckingCol2'])

df.show()

+------+-----+-----------+------------+
TranID|RevID|CheckingCol|CheckingCol2|
+------+-----+-----------+------------+
 1| 1| ABC| ABC|
 2| 1| ABC| ABC|
 3| 6| ABCDE| ABCDE|
 4| 3| ABCDE| ABC|
 5| 7| ABCDE| ABC|
+------+-----+-----------+------------+

In [12]:
from pyspark.sql.functions import *
df.withColumn("TotalMatch", when((col("RevID").contains(col("TranID"))) & (col("CheckingCol") == col("CheckingCol2")), lit("Yes")).otherwise(lit("No"))).show()

+------+-----+-----------+------------+----------+
TranID|RevID|CheckingCol|CheckingCol2|TotalMatch|
+------+-----+-----------+------------+----------+
 1| 1| ABC| ABC| Yes|
 2| 1| ABC| ABC| No|
 3| 6| ABCDE| ABCDE| No|
 4| 3| ABCDE| ABC| No|
 5| 7| ABCDE| ABC| No|
+------+-----+-----------+------------+----------+

In [13]:
list=[[1,40],
     [1,30],
     [2,10],
     [2,90],
     [3,20],
     [3,10],
     [4,2],
     [4,5]]

df=spark.createDataFrame(list,['id','value'])

df.show()

+---+-----+
 id|value|
+---+-----+
 1| 40|
 1| 30|
 2| 10|
 2| 90|
 3| 20|
 3| 10|
 4| 2|
 4| 5|
+---+-----+

In [14]:
w1=Window().partitionBy("id")
w=Window().partitionBy()
df.withColumn("grouped_total",F.sum("value").over(w1))\
  .withColumn("anti_grouped_total", (F.sum("value").over(w))-F.col("grouped_total"))\
  .groupBy("id").agg(F.first("grouped_total").alias("grouped_total"),\
                     F.first("anti_grouped_total").alias("anti_grouped_total"))\
  .drop("value").orderBy("id").show()

+---+-------------+------------------+
 id|grouped_total|anti_grouped_total|
+---+-------------+------------------+
 1| 70| 137|
 2| 100| 107|
 3| 30| 177|
 4| 7| 200|
+---+-------------+------------------+

In [15]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F
w = Window().partitionBy()
df.groupBy("id").agg(F.sum("value").alias("grouped_total"))\
          .withColumn("anti_grouped_total",F.sum("grouped_total").over(w)-F.col("grouped_total")).show()

+---+-------------+------------------+
 id|grouped_total|anti_grouped_total|
+---+-------------+------------------+
 1| 70| 137|
 3| 30| 177|
 2| 100| 107|
 4| 7| 200|
+---+-------------+------------------+

In [16]:
+---+-------------+------------------+
| id|grouped_total|anti_grouped_total|
+---+-------------+------------------+
|  1|           70|               137|
|  2|          100|               107|
|  3|           30|               177|
|  4|            7|               200|
+---+-------------+------------------+

In [17]:
from pyspark.sql import functions as F
df.withColumn("anti_id_1", F.when(F.col("id")==1, F.lit('1')).otherwise(F.lit('Not_1')))\
  .groupBy("anti_id_1").agg(F.sum("value").alias("sum")).show()

+---------+---+
anti_id_1|sum|
+---------+---+
 1| 70|
 Not_1|137|
+---------+---+

In [18]:
list=[['pen',10],
      ['book',40],
      ['bottle',80],
      ['glass',55]]

df=spark.createDataFrame(list,['product','cost'])


df.show()

+-------+----+
product|cost|
+-------+----+
 pen| 10|
 book| 40|
 bottle| 80|
 glass| 55|
+-------+----+

In [19]:
df.withColumn("JSON", F.to_json(F.create_map(F.lit("product"),F.col("product"),F.lit("cost"),F.col("cost")))).drop("product","cost").show(truncate=False)

+--------------------------------+
JSON |
+--------------------------------+
{"product":"pen","cost":"10"} |
{"product":"book","cost":"40"} |
{"product":"bottle","cost":"80"}|
{"product":"glass","cost":"55"} |
+--------------------------------+

In [20]:
list=[[1582749601000]]

df=spark.createDataFrame(list,['millis'])

df.show()


+-------------+
 millis|
+-------------+
1582749601000|
+-------------+

In [21]:
df.withColumn("as_date",F.from_unixtime(F.col("millis")/1000)).show()

+-------------+-------------------+
 millis| as_date|
+-------------+-------------------+
1582749601000|2020-02-26 20:40:01|
+-------------+-------------------+

In [22]:
df.withColumn('as_date', F.from_unixtime((F.col('millis')/1000))).show()

+-------------+-------------------+
 millis| as_date|
+-------------+-------------------+
1582749601000|2020-02-26 20:40:01|
+-------------+-------------------+

In [23]:
list1=[['test1','20','ls',10],
      ['test2', '','baz',15],
      ['test3', '','az',19]]

df1=spark.createDataFrame(list1,['Name','Age','Address','Id'])

list2=[['test4','20','bas',10],
       ['test5','','baz',25],
       ['test6','40','az',19]]

df2=spark.createDataFrame(list2,['Name','Age','Address','Id'])

df1.show()

df2.show()


+-----+---+-------+---+
 Name|Age|Address| Id|
+-----+---+-------+---+
test1| 20| ls| 10|
test2| | baz| 15|
test3| | az| 19|
+-----+---+-------+---+

+-----+---+-------+---+
 Name|Age|Address| Id|
+-----+---+-------+---+
test4| 20| bas| 10|
test5| | baz| 25|
test6| 40| az| 19|
+-----+---+-------+---+

In [24]:
list1=[[[1, 2, 3, 4, 5, 6],[3.42102562348192E-6, 4.2159323917750995E-6, 3.924587540944015E-6, 4.167182871752131E-6, 4.109192066532302E-6, 4.297804458327455E-6]],
[[1, 2, 3, 4, 5, 6],[1.384402399630826E-5, 9.913141993957704E-6, 1.1145077060247102E-5, 1.1005472165326649E-5, 1.1004462921073546E-5, 1.1004462921073546E-5]]]
df=spark.createDataFrame(list1,['x','y'])

display(df)

x,y
"List(1, 2, 3, 4, 5, 6)","List(3.42102562348192E-6, 4.2159323917750995E-6, 3.924587540944015E-6, 4.167182871752131E-6, 4.109192066532302E-6, 4.297804458327455E-6)"
"List(1, 2, 3, 4, 5, 6)","List(1.384402399630826E-5, 9.913141993957704E-6, 1.1145077060247102E-5, 1.1005472165326649E-5, 1.1004462921073546E-5, 1.1004462921073546E-5)"


In [25]:
from pyspark.sql import functions as F
from pyspark.sql.type import *
import numpy as np
def get_slope_func(x,y,order=1):
    coeffs = np.polyfit(x, y, order)
    slope = coeffs[-2]
    return float(slope)
get_slope = F.udf(get_slope_func, returnType=DoubleType())

df.select(get_slope(F.col("x"), F.col("y")).alias("slope")).show(truncate=False)

+-----+
slope|
+-----+
null |
null |
+-----+

In [26]:
╔═══════╦═════╦═════════╦════╗
║ Name  ║ Age ║ Address ║ Id ║
╠═══════╬═════╬═════════╬════╣
║ test1 ║ 20  ║ ls      ║ 10 ║
╠═══════╬═════╬═════════╬════╣
║ test2 ║ 40  ║  az     ║ 19 ║
╚═══════╩═════╩═════════╩════╝

In [28]:
df = spark.createDataFrame([(1,10,"a"),(3,2,"a"),(1,2,"b"),(2,5,"a"),(2,1,"b"),(9,0,"b"),(4,1,"b"),(7,8,"a"),(3,8,"b"),(2,5,"a"),(0,0,"a"),(4,3,"a")],["time", "value", "class"] )
df.orderBy("class","time").show()

+----+-----+-----+
time|value|class|
+----+-----+-----+
 0| 0| a|
 1| 10| a|
 2| 5| a|
 2| 5| a|
 3| 2| a|
 4| 3| a|
 7| 8| a|
 1| 2| b|
 2| 1| b|
 3| 8| b|
 4| 1| b|
 9| 0| b|
+----+-----+-----+

In [29]:
w1=Window().partitionBy("class").orderBy("time","value")
w2=Window().partitionBy("class").orderBy('rownum')
df.withColumn('rownum', F.row_number().over(w1))\
  .withColumn('cumsum_value', F.sum("value").over(w2)).drop('rownum').show()

+----+-----+-----+------------+
time|value|class|cumsum_value|
+----+-----+-----+------------+
 1| 2| b| 2|
 2| 1| b| 3|
 3| 8| b| 11|
 4| 1| b| 12|
 9| 0| b| 12|
 0| 0| a| 0|
 1| 10| a| 10|
 2| 5| a| 15|
 2| 5| a| 20|
 3| 2| a| 22|
 4| 3| a| 25|
 7| 8| a| 33|
+----+-----+-----+------------+

In [30]:
list1=[[3,'7/1/2019','Current'],
       [1,'6/9/2019','Expired'],
       [1,'1/1/2019','Current']]

data=spark.createDataFrame(list1,['USER','DATE','USER_STATE'])

list2=[[1,'7/1/2018',10.00],
       [1,'5/1/2019',40.00],
       [1,'2/2/2019',10.00],
       [3,'1/2/2019',15.00]]

aggregation=spark.createDataFrame(list2,['USER','CHARGEDATE','AMOUNTPAID'])

data.show()
aggregation.show()


+----+--------+----------+
USER| DATE|USER_STATE|
+----+--------+----------+
 3|7/1/2019| Current|
 1|6/9/2019| Expired|
 1|1/1/2019| Current|
+----+--------+----------+

+----+----------+----------+
USER|CHARGEDATE|AMOUNTPAID|
+----+----------+----------+
 1| 7/1/2018| 10.0|
 1| 5/1/2019| 40.0|
 1| 2/2/2019| 10.0|
 3| 1/2/2019| 15.0|
+----+----------+----------+

In [31]:
data.join(aggregation,['USER'])\
      .withColumn("DATE",F.to_date("DATE","M/d/yyyy"))\
      .withColumn("CHARGEDATE", F.to_date("CHARGEDATE", "M/d/yyyy"))\
      .filter("DATE>CHARGEDATE")\
      .groupBy("USER","DATE","USER_STATE").agg(F.mean("AMOUNTPAID").alias("mean_amount_paid"))\
      .show()

+----+----------+----------+----------------+
USER| DATE|USER_STATE|mean_amount_paid|
+----+----------+----------+----------------+
 1|2019-06-09| Expired| 20.0|
 1|2019-01-01| Current| 10.0|
 3|2019-07-01| Current| 15.0|
+----+----------+----------+----------------+

In [32]:
table1.createOrReplaceTempView("data")
table2.createOrReplaceTempView("aggregation")

In [33]:
spark.sql("""select d.user, date, user_state, avg(amountpaid) as mean_amount_paid from data d join aggregation a on d.user = a.user where d.date > a.chargedate group by d.user, date, user_state""").show()

+----+--------+----------+----------------+
user| date|user_state|mean_amount_paid|
+----+--------+----------+----------------+
 1|6/9/2019| Expired| 25.0|
 3|7/1/2019| Current| 15.0|
+----+--------+----------+----------------+

In [34]:
+---+---------- +----------+---------------          
|USER|   DATE   |USER_STATE|MEAN_AMOUNTPAID|   
+---+---------- +----------+--------------- 
|  3 | 7/1/2019 |  Current |    15.00      |
|  1 | 6/9/2019 |  Expired |    20.00      | 
|  1 | 1/1/2019 |  Current |    10.00      |
+----+----------+----------+--------------- 